# Распознавание текста

## CRNN+CTC loss baseline

В данном ноутбуке представлен baseline модели распознавания текста с помощью CRNN модели и CTC loss. Вы можете добавить новые аугментации или изменить структуру данной модели, или же попробовать совершенно новую архитектуру.

# 0. Установка и подгрузука библиотек

Установка библиотек, под которым запускается данный бейзлайн.

In [1]:
!pip install gdown

!gdown --id 1t2Wm_nqy2d198adMmZpSRYm_0mDQXkIs

!unzip -q colab-image.zip -x "__MACOSX/*"

!mv colab-image/* .

!rm -r -f colab-image/ colab-image.zip sample_data/

!pip install mlflow boto3

!pip install albumentations

# !pip install numpy==1.20.3
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install opencv-python==4.5.2.52
# !pip install matplotlib==3.4.2

  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ | done
  Preparing metadata (pyproject.toml) ... - \ | done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14775 sha256=b2f416f87d745ef53af10fd581dd251b1effbf86ef56938533397ddb754bb349
  Stored in directory: /root/.cache/pip/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1t2Wm_nqy2d198adMmZpSRYm_0mDQXkIs
To: /kaggle/working/colab-image.zip
100%|███████████████████████████████████████| 4.67G/4.67G [00:19<00:00, 234MB/s]
     |████████████████████████████████| 16.5 MB 38 kB/s             
     |████████████████████████████████| 58 kB

In [2]:
!ls

__notebook__.ipynb  data  mlflowcred.py


In [3]:
!nvidia-smi

Wed Mar  2 10:12:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!nproc

2


In [5]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence

from torchvision.transforms import transforms

import numpy as np
import cv2
import os
import json
from matplotlib import pyplot as plt
from random import shuffle, randint, random, seed

import urllib3
urllib3.disable_warnings()

from traceback import format_exc

import mlflowcred
from mlflow import mlflow, log_metric, log_param, log_params, log_artifact, log_artifacts, log_dict, log_text, set_tag
from mlflow.tracking import MlflowClient

client = MlflowClient()

from tqdm.notebook import tqdm

import albumentations as A

In [6]:
seed(179)
torch.manual_seed(179)
np.random.seed(179)

## 2. Зададим параметры обучения

Здесь мы можем поправить конфиги обучения - задать размер батча, количество эпох, размер входных изображений, а также установить пути к датасетам.

In [7]:
config_json = {
    "alphabet": ''' !"%\'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPRSTUVWXY[]_abcdefghijklmnopqrstuvwxyz|}ЁАБВГДЕЖЗИКЛМНОПРСТУФХЦЧШЩЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё№''',
    "image": {
        "width": 256,
        "height": 32
    },
    "train": {
        "root_path": "data/images/",
        "json_path": "data/train_labels_splitted.json",
        "batch_size": 440
    },
    "val": {
        "root_path": "data/images/",
        "json_path": "data/val_labels_splitted.json",
        "batch_size": 1450
    },
    'data_loader': {
        'num_workers': 2,
    },
    'training': {
        'experiment_name': "final-baseline",
        'run_name': None,  # 'baseline + 50 epoch',
        'continue_run_id': 'b8e34a84cd2141c987871ea2810c42ea',
        'continue_from': 's3://mlflow/13/b8e34a84cd2141c987871ea2810c42ea/artifacts/model-final-4.8473-82.3451',
        'start_epoch': 50,
        "last_epoch": 100,
        'base_lr': 5e-4,
#         'max_lr': 5e-4,
#         'warmup_epochs': 10,
        'optimizer_patience': 15,
        'optimizer_factor': 0.5,
        "save_dir": "output/"
    }
}

In [8]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

## 3. Теперь определим класс датасета (torch.utils.data.Dataset) и другие вспомогательные функции

In [9]:
# функция которая помогает объединять картинки и таргет-текст в батч
def collate_fn(batch):
    images, texts, enc_texts = zip(*batch)
    images = torch.stack(images, 0)
    text_lens = torch.LongTensor([len(text) for text in texts])
    enc_pad_texts = pad_sequence(enc_texts, batch_first=True, padding_value=0)
    return images, texts, enc_pad_texts, text_lens


def get_data_loader(
    transforms, json_path, root_path, tokenizer, batch_size, drop_last
):
    dataset = OCRDataset(json_path, root_path, tokenizer, transforms)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        collate_fn=collate_fn,
        batch_size=batch_size,
        num_workers=config_json['data_loader']['num_workers'],
    )
    return data_loader


class OCRDataset(Dataset):
    def __init__(self, json_path, root_path, tokenizer, transform=None):
        super().__init__()
        self.transform = transform
        with open(json_path, 'r') as f:
            data = json.load(f)
        self.data_len = len(data)

        self.img_paths = []
        self.texts = []
        for img_name, text in data.items():
            self.img_paths.append(os.path.join(root_path, img_name))
            self.texts.append(text)
        self.enc_texts = tokenizer.encode(self.texts)

    def __len__(self):
        return self.data_len

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        text = self.texts[idx]
        enc_text = torch.LongTensor(self.enc_texts[idx])
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, text, enc_text


class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## 4. Здесь определен Токенайзер - вспопогательный класс, который преобразует текст в числа

Разметка-текст с картинок преобразуется в числовое представление, на которых модель может учиться. Также может преобразовывать числовое предсказание модели обратно в текст.

In [10]:
OOV_TOKEN = '<OOV>'
CTC_BLANK = '<BLANK>'


def get_char_map(alphabet):
    """Make from string alphabet character2int dict.
    Add BLANK char fro CTC loss and OOV char for out of vocabulary symbols."""
    char_map = {value: idx + 2 for (idx, value) in enumerate(alphabet)}
    char_map[CTC_BLANK] = 0
    char_map[OOV_TOKEN] = 1
    return char_map


class Tokenizer:
    """Class for encoding and decoding string word to sequence of int
    (and vice versa) using alphabet."""

    def __init__(self, alphabet):
        self.char_map = get_char_map(alphabet)
        self.rev_char_map = {val: key for key, val in self.char_map.items()}

    def encode(self, word_list):
        """Returns a list of encoded words (int)."""
        enc_words = []
        for word in word_list:
            enc_words.append(
                [self.char_map[char] if char in self.char_map
                 else self.char_map[OOV_TOKEN]
                 for char in word]
            )
        return enc_words

    def get_num_chars(self):
        return len(self.char_map)

    def decode(self, enc_word_list):
        """Returns a list of words (str) after removing blanks and collapsing
        repeating characters. Also skip out of vocabulary token."""
        dec_words = []
        for word in enc_word_list:
            word_chars = ''
            for idx, char_enc in enumerate(word):
                # skip if blank symbol, oov token or repeated characters
                if (
                    char_enc != self.char_map[OOV_TOKEN]
                    and char_enc != self.char_map[CTC_BLANK]
                    # idx > 0 to avoid selecting [-1] item
                    and not (idx > 0 and char_enc == word[idx - 1])
                ):
                    word_chars += self.rev_char_map[char_enc]
            dec_words.append(word_chars)
        return dec_words

## 5. Accuracy в качестве метрики

Accuracy измеряет долю предсказанных строк текста, которые полностью совпадают с таргет текстом.

In [11]:
def string_accuracy(pred_texts, gt_texts):
    assert len(pred_texts) == len(gt_texts)
    correct = 0
    for pred_text, gt_text in zip(pred_texts, gt_texts):
        correct += int(pred_text == gt_text)
    return 100 * correct / len(gt_texts)


def levenshtein_distance(first, second):
    distance = [[0 for _ in range(len(second) + 1)]
                for _ in range(len(first) + 1)]
    for i in range(len(first) + 1):
        for j in range(len(second) + 1):
            if i == 0:
                distance[i][j] = j
            elif j == 0:
                distance[i][j] = i
            else:
                diag = distance[i - 1][j - 1] + (first[i - 1] != second[j - 1])
                upper = distance[i - 1][j] + 1
                left = distance[i][j - 1] + 1
                distance[i][j] = min(diag, upper, left)
    return distance[-1][-1]


def cer(pred_texts, gt_texts):
    assert len(pred_texts) == len(gt_texts)
    lev_distances, num_gt_chars = 0, 0
    for pred_text, gt_text in zip(pred_texts, gt_texts):
        lev_distances += levenshtein_distance(pred_text, gt_text)
        num_gt_chars += len(gt_text)
    
    return 100 * lev_distances / num_gt_chars

## 6. Аугментации

Здесь мы задаем базовые аугментации для модели. Вы можете написать свои или использовать готовые библиотеки типа albumentations

In [12]:
class Normalize:
    def __call__(self, img):
        img = img.astype(np.float32) / 255
        return img


class ToTensor:
    def __call__(self, arr):
        arr = torch.from_numpy(arr)
        return arr


class MoveChannels:
    """Move the channel axis to the zero position as required in pytorch."""

    def __init__(self, to_channels_first=True):
        self.to_channels_first = to_channels_first

    def __call__(self, image):
        if self.to_channels_first:
            return np.moveaxis(image, -1, 0)
        else:
            return np.moveaxis(image, 0, -1)


class ImageResize:
    def __init__(self, height, width):
        self.height = height
        self.width = width

    def __call__(self, image):
#         w1 = min(self.width, int(image.shape[1] / image.shape[0] * self.height))
        txt = cv2.resize(image, (self.width, self.height), interpolation=cv2.INTER_AREA)
#         bg = np.zeros((self.height, self.width, 3)).astype(int)
#         bg[:txt.shape[0], :txt.shape[1]] = txt
        
        return txt

class RandomTransform:
    def __init__(self):
        self.transform = A.Compose([
            A.RGBShift(p=0.5),
            A.ColorJitter(p=0.5),
            A.GaussNoise(p=0.5),
            A.Rotate(limit=4),
            A.GridDistortion(p=0.5),
        ])
    
    def __call__(self, image):
        return self.transform(image=image)["image"]


def get_train_transforms(height, width):
    transforms = torchvision.transforms.Compose([
        RandomTransform(),
        ImageResize(height, width),
        MoveChannels(to_channels_first=True),
        Normalize(),
        ToTensor()
    ])
    return transforms


def get_val_transforms(height, width):
    transforms = torchvision.transforms.Compose([
        ImageResize(height, width),
        MoveChannels(to_channels_first=True),
        Normalize(),
        ToTensor()
    ])
    return transforms

## 7. Здесь определяем саму модель

In [13]:
def get_resnet34_backbone(pretrained=True):
    m = torchvision.models.resnet34(pretrained=True)
    input_conv = nn.Conv2d(3, 64, 7, 1, 3)
    blocks = [input_conv, m.bn1, m.relu,
              m.maxpool, m.layer1, m.layer2, m.layer3]
    return nn.Sequential(*blocks)


class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers,
            dropout=dropout, batch_first=True, bidirectional=True)

    def forward(self, x):
        out, _ = self.lstm(x)
        return out


class CRNN(nn.Module):
    def __init__(
        self, number_class_symbols, time_feature_count=256, lstm_hidden=256,
        lstm_len=2,
    ):
        super().__init__()
        self.feature_extractor = get_resnet34_backbone(pretrained=True)
        self.avg_pool = nn.AdaptiveAvgPool2d(
            (time_feature_count, time_feature_count))
        self.bilstm = BiLSTM(time_feature_count, lstm_hidden, lstm_len)
        self.classifier = nn.Sequential(
            nn.Linear(lstm_hidden * 2, time_feature_count),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(time_feature_count, number_class_symbols)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        b, c, h, w = x.size()
        x = x.view(b, c * h, w)
        x = self.avg_pool(x)
        x = x.transpose(1, 2)
        x = self.bilstm(x)
        x = self.classifier(x)
        x = nn.functional.log_softmax(x, dim=2).permute(1, 0, 2)
        return x

## 8. Переходим к самому скрипту обучения - циклы трейна и валидации

In [14]:
def predict(images, model, tokenizer, device):
    model.eval()
    images = images.to(device)
    with torch.no_grad():
        output = model(images)
    pred = torch.argmax(output.detach().cpu(), -1).permute(1, 0).numpy()
    text_preds = tokenizer.decode(pred)
    return text_preds


def val_loop(data_loader, model, tokenizer, device, epoch=-1):
    acc_avg = AverageMeter()
    cer_avg = AverageMeter()

    pbar = tqdm(data_loader, desc='Validating')
    for images, texts, _, _ in pbar:
        batch_size = len(texts)
        text_preds = predict(images, model, tokenizer, device)
        
        # gpu memory
        al_m = round(torch.cuda.memory_allocated(0) / 1024 ** 3, 2)
        cache_m = round(torch.cuda.memory_reserved(0) / 1024 ** 3, 2)
        
        acc_avg.update(string_accuracy(text_preds, texts), batch_size)
        cer_avg.update(cer(text_preds, texts), batch_size)

        pbar.set_postfix({'acc': acc_avg.avg, 'cer': cer_avg.avg, 'allocated memory': al_m, 'cache memory': cache_m})     

    
    log_metric('val_acc', acc_avg.avg, step=epoch)
    log_metric('val_cer', cer_avg.avg, step=epoch)

    print(f'Validation, acc: {acc_avg.avg:.4f}%, cer: {cer_avg.avg:.4f}%')

    return acc_avg.avg, cer_avg.avg


In [15]:
def train_loop(data_loader, model, criterion, optimizer, epoch):
    al_m = AverageMeter()
    cache_m = AverageMeter()

    loss_avg = AverageMeter()
    model.train()
    pbar = tqdm(data_loader, desc=f'Epoch {epoch}')
    for images, texts, enc_pad_texts, text_lens in pbar:
        model.zero_grad()
        images = images.to(DEVICE)

        # gpu memory
        now_al_m = round(torch.cuda.memory_allocated(0) / 1024 ** 3, 2)
        now_cache_m = round(torch.cuda.memory_reserved(0) / 1024 ** 3, 2)
        al_m.update(now_al_m, 1)
        cache_m.update(now_cache_m, 1)

        batch_size = len(texts)
        
#         with torch.cuda.amp.autocast():
        output = model(images)

        output_lenghts = torch.full(
            size=(output.size(1),),
            fill_value=output.size(0),
            dtype=torch.long
        )

        loss = criterion(output, enc_pad_texts, output_lenghts, text_lens)

        pbar.set_postfix({'loss': loss.item(), 'allocated memory': now_al_m, 'cache memory': now_cache_m})

        loss_avg.update(loss.item(), batch_size)
#         log_metric('loss', loss.item())
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 2)
        optimizer.step()


    for param_group in optimizer.param_groups:
        lr = param_group['lr']
    
    log_metric('loss', loss_avg.avg, step=epoch)
    log_metric('lr', lr, step=epoch)

    log_metric('train_allocated_memory_gb', al_m.avg, step=epoch)
    log_metric('train_cached_memory_gb', cache_m.avg, step=epoch)

    print(f'\nEpoch {epoch}, Loss: {loss_avg.avg:.5f}, LR: {lr:.7f}')
    return loss_avg.avg


def get_loaders(tokenizer, config):
    train_transforms = get_train_transforms(
        height=config['image']['height'],
        width=config['image']['width']
    )
    train_loader = get_data_loader(
        json_path=config['train']['json_path'],
        root_path=config['train']['root_path'],
        transforms=train_transforms,
        tokenizer=tokenizer,
        batch_size=config['train']['batch_size'],
        drop_last=True
    )
    val_transforms = get_val_transforms(
        height=config['image']['height'],
        width=config['image']['width']
    )
    val_loader = get_data_loader(
        transforms=val_transforms,
        json_path=config['val']['json_path'],
        root_path=config['val']['root_path'],
        tokenizer=tokenizer,
        batch_size=config['val']['batch_size'],
        drop_last=False
    )
    return train_loader, val_loader


def train(config):
    tokenizer = Tokenizer(config['alphabet'])
    os.makedirs(config['training']['save_dir'], exist_ok=True)
    train_loader, val_loader = get_loaders(tokenizer, config)

    model = CRNN(number_class_symbols=tokenizer.get_num_chars())
    if config_json['training']['continue_from'] is not None:
        model.load_state_dict(
            mlflow.pytorch.load_state_dict(config_json['training']['continue_from'])
    )
    log_text(str(model), 'model-struct.txt')
    model.to(DEVICE)

    criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=config['training']['base_lr'],
        weight_decay=0.01
    )
#     optimizer = torch.optim.SGD(model.parameters(), lr=config['training']['base_lr'], momentum=0.9, weight_decay=1e-2)
    log_text(str(optimizer), 'optimizer.txt')
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=optimizer,
        mode='min',
        factor=config['training']['optimizer_factor'],
        patience=config['training']['optimizer_patience'],
        verbose=True
    )
#     scheduler = torch.optim.lr_scheduler.CyclicLR(
#                 optimizer, 
#                 base_lr=config['training']['base_lr'], max_lr=config['training']['max_lr'],
#                 cycle_momentum=True,
#                 step_size_up=len(train_loader) * config['training']['warmup_epochs'],
#                 step_size_down=len(train_loader) * (config['training']['last_epoch'] - config['training']['start_epoch'] - config['training']['warmup_epochs']),
#                 verbose=True
#     )
#     log_dict(scheduler.state_dict(), 'scheduler.json')
    
    best_acc, best_cer = val_loop(val_loader, model, tokenizer, DEVICE)
    for epoch in tqdm(range(config['training']['start_epoch'], config['training']['last_epoch'])):
        print('-' * 150)
        
        loss_avg = train_loop(train_loader, model, criterion, optimizer, epoch)
        acc_avg, cer_avg = val_loop(val_loader, model, tokenizer, DEVICE, epoch)
        scheduler.step(cer_avg)
        
        if cer_avg < best_cer:
            best_cer = cer_avg
            mlflow.pytorch.log_state_dict(model.state_dict(), f'model-{epoch:0>3}-{cer_avg:.4f}-{acc_avg:.4f}')
            mlflow.pytorch.log_model(model, f'model-{epoch:0>3}-{cer_avg:.4f}-{acc_avg:.4f}')
            print('Model weights saved')
        
        print('-' * 150)
    
    mlflow.pytorch.log_state_dict(model.state_dict(), f'model-final-{cer_avg:.4f}-{acc_avg:.4f}')
    mlflow.pytorch.log_model(model, f'model-final-{cer_avg:.4f}-{acc_avg:.4f}')

    return model


def start_training(config):
    torch.cuda.empty_cache()

    mlflow.set_experiment(config_json['training']['experiment_name'])
    
    model = None

    with mlflow.start_run(run_name=config_json['training']['run_name'], run_id=config_json['training']['continue_run_id']) as run:    
        log_dict(config_json, 'config_json.json')

        log_param('image.width', config_json['image']['width'])
        log_param('image.height', config_json['image']['height'])

        log_param('train.batch_size', config_json['train']['batch_size'])
        log_param('val.batch_size', config_json['val']['batch_size'])
        
        try:
            model = train(config)
        except Exception as error:
            log_text(format_exc(), 'error.txt')
            raise Exception(repr(error))
    
    return model


## 9. Запускаем обучение!

In [16]:
model = start_training(config_json)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Validating:   0%|          | 0/28 [00:00<?, ?it/s]

Validation, acc: 82.3451%, cer: 4.8473%


  0%|          | 0/50 [00:00<?, ?it/s]

...

Validation, acc: 81.1899%, cer: 5.1884%
------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------


Epoch 99:   0%|          | 0/276 [00:00<?, ?it/s]


Epoch 99, Loss: 0.12155, LR: 0.0002500


Validating:   0%|          | 0/28 [00:00<?, ?it/s]

Validation, acc: 81.2764%, cer: 5.1764%
------------------------------------------------------------------------------------------------------------------------------------------------------


In [17]:
!rm -r -f data/ mlflowcred.py